# CS 237: Homework 6 Programming Portion

In [1]:
# add your imports here

# here are some examples of imports
import matplotlib.pyplot as plt   # for plotting
import numpy as np                # for simulating random choices
import bisect as bi               # for binary search on sorted lists

# Exercise 1:  Data structures for discrete RVs:  PDFs and CDFs

In this exercise, we'll build simple data structures to represent PDFs and CDFs of discrete random variables (RVs). 

As input, the user will provide a dictionary of values of a discrete RV and their associate percentage probabilities.  At the end of this notebook are some test cases involving frequencies of English letters in documents and Super Bowl win probabilities of NFL teams.  You're welcome and encouraged to try others, including larger datasets.

Concretely, the input is a Python dictionary with tuples of the form (RV_value: percentage_prob)

For example, if the user creates an RV named X with '{4: 50.0, 22: 30.0, 2: 20.0}', we will want to represent the following information (in sorted order), sorted by values:

    PDF  f(X):  Pr[X = 2] = 0.2,  Pr[X = 4] = 0.5,  Pr[X = 22] = 0.3
    CDF  F(X):  Pr[X < 2] = 0,  Pr[2 <=X < 4] = 0.2,   Pr[4 <= X < 22] = 0.7,   Pr[22 <= X] = 1 

We also need to check that the user has specified a valid PDF (probabilities sums to 1).

To manipulate PDFs and CDFs efficiently, we will store the values in sorted order.  To be space-efficient, we only need to store those values where the PDF has mass, or equivalently, where the CDF has a discontinuity.  So our random variable X can be described by:

    values:       2     4     22
    --------------------------------
    PDF f()     0.2   0.5    0.3 
    CDF F()     0.2   0.7    1.0       
    
    # note that F() always starts at zero, so we don't need to store that explicitly
===

# Exercise 1a) Complete the implementation of the __init__ method of the RV class below    

In [2]:
class RV:

    #  Create an RV from a user-specified dictionary.  See examples of input below.
    #  Member variables are
    #       self.values  (sorted list)     
    #       self.PDF     (list of probabilities)
    #       self.CDF     (list of cumulative probabilities)
    #
    def __init__(self, dict):
        
        #   create sorted list of values
        self.values = []
        for x in dict:
            self.values.append(x)
            self.values.sort()            # create a sorted list of outcomes
    
        #   create PDF
        self.PDF = [None] * len(self.values)
        for i in range(len(self.values)):
            pct = dict[self.values[i]]
            self.PDF[i] = round(pct * 10000)/1000000   
            # round to 8 decimal digits to avoid numerical precision problems
            # see what bad things happen if you just use pct/100
    
 
        # compute F_X from f_X
        self.CDF = [None] * len(self.values)
    
        ######## YOUR CODE HERE
        for i in range(len(self.values)):
            cdfs = 0
            for j in range(i + 1):
                cdfs += self.PDF[j]
            self.CDF[i] = round(cdfs * 10000)/10000 
            # round the numbers 
            
        if self.CDF[len(self.values) - 1] != 1.00:
            self.CDF = None

# Exercise 1b) Complete the implementation of prob() below    

In [3]:
#  Find the leftmost index i in sorted list a such that a[i] == x
def index(a, x):
    
    i = bi.bisect_left(a, x)
    if i != len(a) and a[i] == x:
        return i
    
    return None

  
#  Given an RV value return its likelihood: Pr[R=value]
def prob (R, value):
    pass

###### YOUR CODE HERE
#  We expect you to call index() above
    i = index(R.values, value)
    if i == None:
        return None
    else:
        return R.PDF[i]

# Exercise 1c) Complete the implementation of rangeprob() below    

In [4]:
#  Find the leftmost index in sorted list A strictly greater than x
def find_gt(a, x):
    
    i = bi.bisect_right(a, x)
    if i != len(a):
        return i
    
    return None


#  Find the rightmost index in sorted list A less than or equal to x
def find_le(a, x):
    
    i = bi.bisect_right(a, x)
    if i:
        return i-1
    
    return None


# STUDENTS TO IMPLEMENT
#  Given a range of outcomes specified by an interval: [left, right] and an RV, return Pr[left <= x < right]
def rangeprob (R, left, right):

    pass
###### YOUR CODE HERE
#  We expect you to call find_gt() and find_le() above
    index_right = find_gt(R.values, right)
    index_left = find_le(R.values, left)
    if index_right == 0:
        return 0
    elif index_left == len(R.values) - 1:
        return 1
    elif index_right <= index_left:
        return None
    else:
        return R.CDF[index_right]


# Random Sampling

To generate a random sample from a distribution, it is easiest to use the CDF. 

Here's pseudocode of the method you will implement:

     - Generate a random number y from [0, 1).
     - Search the sorted list of the CDF to find the smallest outcome l such that y < F(l)
         (note that there will always be one such l provided F is a valid CDF)
     - return l
    
Intuitively, y serves to pick a random offset into the CDF, which maps onto an interval 
within the CDF associated with one value of the random variable.The length of that interval is exactly to the probability of the associated value!

For example, with the CDF of our previous random variable
    
    F_X =  [2: 0.2], [4: 0.7], [22, 1.0]
    
We might select y = 0.78.  The smallest value l satisfying y < F[l] is 
    l = 22, F[l] = 1.0
    
So we return '22'.

Alternatively, if we selected y = 0.152, we would return '2'.



# Exercise 1d) Complete the implementation of sample() below    

In [15]:
# STUDENTS TO IMPLEMENT
#  Generate a random sample from a random variable R
def sample(R):
 
    # return "Patriots"   # nonsense placeholder to make notebook interpretable.  Delete
##### Your code here
# Despite the long description, this is only a few lines of code.
# We expect you to use find_gt() again.
    y = 0
    i = 0
    
    y = np.random.rand()
    i = find_gt(R.CDF, y)    
    return R.values[i]

# feel free to insert your own test cases here

# Exercise 2)  Run our test cases on your code.

In [16]:
#  English letter frequencies as a dictionary
letterFrequency = {'E' : 12.0,
'T' : 9.10,
'A' : 8.12,
'O' : 7.68,
'I' : 7.31,
'N' : 6.95,
'S' : 6.28,
'R' : 6.02,
'H' : 5.92,
'D' : 4.32,
'L' : 3.98,
'U' : 2.88,
'C' : 2.71,
'M' : 2.61,
'F' : 2.30,
'Y' : 2.11,
'W' : 2.09,
'G' : 2.03,
'P' : 1.82,
'B' : 1.49,
'V' : 1.11,
'K' : 0.69,
'X' : 0.17,
'Q' : 0.12,
'J' : 0.11,
'Z' : 0.08 }

# Test initialization
Letter = RV(letterFrequency)
print (Letter.values)
print (Letter.PDF)
print (Letter.CDF)

# Test prob() and rangeprob()
print ("Pr[Letter = 'Z'] = " + str(prob(Letter, 'Z')))
print ("Pr[Letter = 'CS237'] = " + str(prob(Letter, 'CS237')))        # This should return None (not a valid value of the RV)
print ("Pr['M' <= Letter < 'X']= " + str(rangeprob(Letter, 'M', 'X')))
print ("Pr['X' <= Letter < 'M']= " + str(rangeprob(Letter, 'X', 'M')))  # This should return None or raise an error

# Test sample()
## Let's see if letter frequencies are enough to produce any reasonable 5-letter English words
for i in range (51):
    word = (""+ sample(Letter)+sample(Letter)+sample(Letter)+sample(Letter)+sample(Letter))
    print (word)



['A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z']
[0.0812, 0.0149, 0.0271, 0.0432, 0.12, 0.023, 0.0203, 0.0592, 0.0731, 0.0011, 0.0069, 0.0398, 0.0261, 0.0695, 0.0768, 0.0182, 0.0012, 0.0602, 0.0628, 0.091, 0.0288, 0.0111, 0.0209, 0.0017, 0.0211, 0.0008]
[0.0812, 0.0961, 0.1232, 0.1664, 0.2864, 0.3094, 0.3297, 0.3889, 0.462, 0.4631, 0.47, 0.5098, 0.5359, 0.6054, 0.6822, 0.7004, 0.7016, 0.7618, 0.8246, 0.9156, 0.9444, 0.9555, 0.9764, 0.9781, 0.9992, 1.0]
Pr[Letter = 'Z'] = 0.0008
Pr[Letter = 'CS237'] = None
Pr['M' <= Letter < 'X']= 0.9992
Pr['X' <= Letter < 'M']= None
NDRDN
FROIS
CAEEU
NTOUA
NALNE
AOREN
OAWIR
HTLOS
EEHRG
IDOEO
DEAIW
TECEU
TCRAN
LENTF
RTEUN
IYOTE
FYTND
IKTIL
NETNM
EWHEV
HEDAE
NLNIN
AOCSE
DTSHL
NOTET
EHSNR
DTNOT
OCTNL
EDEOT
EEEGL
DNTTE
TTYAO
ONPNI
YEOZL
EONES
IHPIP
NIDND
TUUIA
GRMDO
LPTFT
UFEOM
OORSB
ORNSB
NNUOS
CAACE
AOARD
NATSS
NHOAR
OAASK
NOSSG
DKLTA


In [17]:
truncatedLetterFrequency = {'E' : 12.0,
'T' : 9.10,
'A' : 8.12,
'O' : 7.68,
'I' : 7.31}

TruncLetter = RV(truncatedLetterFrequency)

## this is hard to deal with well in Python, but we just want you to set the CDF to None  
print (TruncLetter.CDF)


None


In [18]:
# from fivethirtyeight.com, as of 10/12
SuperBowlWinPercentages = {"Bills": 17,
    "Bucs": 13.9,
    "Ravens": 10.7,
    "Chargers": 8.8,
    "Cardinals": 9,
    "Rams": 8.8,
    "Cowboys": 5,
    "Packers": 4.9,
    "Chiefs": 4,
    "Browns": 3.8,
    "Saints": 3.6,
    "Titans": 2,
    "Seahawks": 0.9,
    "49ers": 0.9,
    "Broncos": 0.8,
    "Bengals": 0.7,
    "Vikings": 0.7,
    "Patriots": 0.7,
    "Colts": 0.7,
    "Raiders": 0.5,
    "Panthers": 0.5,
    "Bears": 0.5,
    "Washington": 0.4,
    "Eagles": 0.4,
    "Steelers": 0.4,
    "Falcons": 0.2,
    "Dolphins": 0.1,
    "Giants": 0.05,
    "Jets": 0.03,
    "Texans": 0.02
}

Winner = RV(SuperBowlWinPercentages)
print (Winner.values)
print (Winner.PDF)
print (Winner.CDF)


while True:
    winner = sample(Winner)
    print (winner)
    if winner == "Patriots":
        break
    


['49ers', 'Bears', 'Bengals', 'Bills', 'Broncos', 'Browns', 'Bucs', 'Cardinals', 'Chargers', 'Chiefs', 'Colts', 'Cowboys', 'Dolphins', 'Eagles', 'Falcons', 'Giants', 'Jets', 'Packers', 'Panthers', 'Patriots', 'Raiders', 'Rams', 'Ravens', 'Saints', 'Seahawks', 'Steelers', 'Texans', 'Titans', 'Vikings', 'Washington']
[0.009, 0.005, 0.007, 0.17, 0.008, 0.038, 0.139, 0.09, 0.088, 0.04, 0.007, 0.05, 0.001, 0.004, 0.002, 0.0005, 0.0003, 0.049, 0.005, 0.007, 0.005, 0.088, 0.107, 0.036, 0.009, 0.004, 0.0002, 0.02, 0.007, 0.004]
[0.009, 0.014, 0.021, 0.191, 0.199, 0.237, 0.376, 0.466, 0.554, 0.594, 0.601, 0.651, 0.652, 0.656, 0.658, 0.6585, 0.6588, 0.7078, 0.7128, 0.7198, 0.7248, 0.8128, 0.9198, 0.9558, 0.9648, 0.9688, 0.969, 0.989, 0.996, 1.0]
Rams
Cardinals
Bills
Chargers
Cardinals
Cardinals
Rams
Bills
Cardinals
Bills
Chargers
Bills
Chargers
Ravens
Titans
Bills
Bucs
Ravens
Cardinals
Saints
Bills
Eagles
Bills
Packers
Chargers
Bears
Bills
Cowboys
Cowboys
Chargers
Bills
Chargers
Panthers
Saints
